<a href="https://colab.research.google.com/github/Amplil/nlp-test/blob/master/opmnt_library_transformer_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 初期化

In [1]:
cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/.shortcut-targets-by-id/1liJWbv2EhngAj7pixTjzRpv9B-Bz9zCt/Colab Notebooks


In [2]:
ls

data@                                      pred.txt
new_colab2.ipynb                           toy-ende@
opmnt_library_test.ipynb                   Transformer@
opmnt_library_transformer_translate.ipynb  transformer_kyoto_ja_en.ipynb
packages@                                  transformer_test.ipynb


In [3]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")
import slacknotice # オリジナルモジュール

In [4]:
pip install OpenNMT-py

     |████████████████████████████████| 216 kB 5.5 MB/s 
     |████████████████████████████████| 16.0 MB 117 kB/s 
     |████████████████████████████████| 73 kB 1.7 MB/s 
     |████████████████████████████████| 56 kB 3.4 MB/s 
     |████████████████████████████████| 1.2 MB 44.1 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [5]:
import torch
import torch.nn as nn
from argparse import Namespace
from collections import defaultdict, Counter

In [6]:
import onmt
from onmt.inputters.inputter import _load_vocab, _build_fields_vocab, get_fields, IterOnDevice
from onmt.inputters.corpus import ParallelCorpus
from onmt.inputters.dynamic_iterator import DynamicDatasetIter
from onmt.translate import GNMTGlobalScorer, Translator, TranslationBuilder
from onmt.utils.misc import set_random_seed

In [7]:
# enable logging
from onmt.utils.logging import init_logger, logger
init_logger()

<RootLogger root (INFO)>

# モデルの読み込み

In [8]:
src_vocab_path = "data/kftt-data-1.0/run/example.vocab.src"
tgt_vocab_path = "data/kftt-data-1.0/run/example.vocab.tgt"

In [9]:
# initialize the frequency counter
counters = defaultdict(Counter)
# load source vocab
_src_vocab, _src_vocab_size = _load_vocab(
    src_vocab_path,
    'src',
    counters)
# load target vocab
_tgt_vocab, _tgt_vocab_size = _load_vocab(
    tgt_vocab_path,
    'tgt',
    counters)

[2021-11-07 06:03:27,318 INFO] Loading src vocabulary from data/kftt-data-1.0/run/example.vocab.src
[2021-11-07 06:03:28,621 INFO] Loaded src vocab has 13637 tokens.
[2021-11-07 06:03:28,635 INFO] Loading tgt vocabulary from data/kftt-data-1.0/run/example.vocab.tgt
[2021-11-07 06:03:28,900 INFO] Loaded tgt vocab has 12199 tokens.


In [10]:
# initialize fields
src_nfeats, tgt_nfeats = 0, 0 # do not support word features for now
fields = get_fields(
    'text', src_nfeats, tgt_nfeats)

In [11]:
# build fields vocab
share_vocab = False
vocab_size_multiple = 1
src_vocab_size = 30000
tgt_vocab_size = 30000
src_words_min_frequency = 1
tgt_words_min_frequency = 1
vocab_fields = _build_fields_vocab(
    fields, counters, 'text', share_vocab,
    vocab_size_multiple,
    src_vocab_size, src_words_min_frequency,
    tgt_vocab_size, tgt_words_min_frequency)

[2021-11-07 06:03:28,971 INFO]  * tgt vocab size: 12203.
[2021-11-07 06:03:28,993 INFO]  * src vocab size: 13639.


In [24]:
src_val = "data/kftt-data-1.0/data/tok/kyoto-test.en"
tgt_val = "data/kftt-data-1.0/data/tok/kyoto-test.ja"

In [25]:
"""
src_data = {"reader": onmt.inputters.str2reader["text"](), "data": src_val}
tgt_data = {"reader": onmt.inputters.str2reader["text"](), "data": tgt_val}
_readers, _data = onmt.inputters.Dataset.config(
    [('src', src_data), ('tgt', tgt_data)])
"""
src_data = {
    "reader": onmt.inputters.str2reader["text"](),
    "data": src_val,
    "features": {}
}
tgt_data = {
    "reader": onmt.inputters.str2reader["text"](),
    "data": tgt_val,
    "features": {}
}
_readers, _data = onmt.inputters.Dataset.config(
    [("src", src_data), ("tgt", tgt_data)]
)

In [15]:
opt.vocab_fields

AttributeError: ignored

In [16]:
opt.src

'data/kftt-data-1.0/data/tok/kyoto-test.en'

In [17]:
opt.shard_size

10000

In [18]:
opt.attn_debug

False

In [ ]:
op

In [19]:
load_test_model(opt)

NameError: ignored

In [26]:
dataset = onmt.inputters.Dataset(
    vocab_fields, readers=_readers, data=_data,
    sort_key=onmt.inputters.str2sortkey["text"])

In [27]:
data_iter = onmt.inputters.OrderedIterator(
            dataset=dataset,
            device="cuda",
            batch_size=10,
            train=False,
            sort=False,
            sort_within_batch=True,
            shuffle=False
        )

In [16]:
src_text_field = vocab_fields["src"].base_field
src_vocab = src_text_field.vocab
src_padding = src_vocab.stoi[src_text_field.pad_token]

tgt_text_field = vocab_fields['tgt'].base_field
tgt_vocab = tgt_text_field.vocab
tgt_padding = tgt_vocab.stoi[tgt_text_field.pad_token]

In [17]:
# LSTMでの翻訳
"""
emb_size = 100
rnn_size = 500
# Specify the core model.

encoder_embeddings = onmt.modules.Embeddings(emb_size, len(src_vocab),
                                             word_padding_idx=src_padding)

encoder = onmt.encoders.RNNEncoder(hidden_size=rnn_size, num_layers=1,
                                   rnn_type="LSTM", bidirectional=True,
                                   embeddings=encoder_embeddings)

decoder_embeddings = onmt.modules.Embeddings(emb_size, len(tgt_vocab),
                                             word_padding_idx=tgt_padding)
decoder = onmt.decoders.decoder.InputFeedRNNDecoder(
    hidden_size=rnn_size, num_layers=1, bidirectional_encoder=True, 
    rnn_type="LSTM", embeddings=decoder_embeddings)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = onmt.models.model.NMTModel(encoder, decoder)
model.to(device)

# Specify the tgt word generator and loss computation module
model.generator = nn.Sequential(
    nn.Linear(rnn_size, len(tgt_vocab)),
    nn.LogSoftmax(dim=-1)).to(device)

loss = onmt.utils.loss.NMTLossCompute(
    criterion=nn.NLLLoss(ignore_index=tgt_padding, reduction="sum"),
    generator=model.generator)
"""

In [18]:
src_reader = onmt.inputters.str2reader["text"]
tgt_reader = onmt.inputters.str2reader["text"]
scorer = GNMTGlobalScorer(alpha=0.7, 
                          beta=0., 
                          length_penalty="avg", 
                          coverage_penalty="none")
gpu = 0 if torch.cuda.is_available() else -1
translator = Translator(model=model, 
                        fields=vocab_fields, 
                        src_reader=src_reader, 
                        tgt_reader=tgt_reader, 
                        global_scorer=scorer,
                        gpu=gpu)
builder = onmt.translate.TranslationBuilder(data=dataset, 
                                            fields=vocab_fields)

In [8]:
import onmt.opts as opts
from onmt.utils.parse import ArgumentParser
#from onmt.opts import dynamic_prepare_opts

parser = ArgumentParser(description='translate.py')
#dynamic_prepare_opts(parser, build_vocab_only=True)
opts.translate_opts(parser)

base_args = (["-model", "data/kftt-data-1.0/run/model_step_15000.pt", "-src", "data/kftt-data-1.0/data/tok/kyoto-test.en", "-output", "data/kftt-data-1.0/out.txt", "-gpu", "0", "-verbose"])
opt, unknown = parser.parse_known_args(base_args)
opt

Namespace(align_debug=False, alpha=0.0, attn_debug=False, avg_raw_probs=False, ban_unk_token=False, batch_size=30, batch_type='sents', beam_size=5, beta=-0.0, block_ngram_repeat=0, coverage_penalty='none', data_type='text', dump_beam='', fp32=False, gpu=0, ignore_when_blocking=[], int8=False, length_penalty='none', log_file='', log_file_level='0', max_length=100, max_sent_length=None, min_length=0, models=['data/kftt-data-1.0/run/model_step_15000.pt'], n_best=1, output='data/kftt-data-1.0/out.txt', phrase_table='', random_sampling_temp=1.0, random_sampling_topk=0, random_sampling_topp=0.0, ratio=-0.0, replace_unk=False, report_align=False, report_time=False, seed=-1, shard_size=10000, src='data/kftt-data-1.0/data/tok/kyoto-test.en', src_feats=None, stepwise_penalty=False, tgt=None, tgt_prefix=False, verbose=True)

In [30]:
from onmt.translate.translator import build_translator

ArgumentParser.validate_translate_opts(opt)
logger = init_logger(opt.log_file)

translator = build_translator(opt, logger=logger, report_score=True)

In [20]:
# out_fileの作成
"""
if out_file is None:
    out_file = codecs.open(opt.output, "w+", "utf-8")
"""
load_test_model = (
    onmt.decoders.ensemble.load_test_model
    if len(opt.models) > 1
    else onmt.model_builder.load_test_model
)
vocab_fields, model, model_opt = load_test_model(opt) # fields->vocab_fields

scorer = onmt.translate.GNMTGlobalScorer.from_opt(opt)

In [9]:
#def load_test_model(opt, model_path=None)
"""
from onmt.utils.parse import ArgumentParser
import onmt.inputters as inputters
from onmt.utils.misc import use_gpu

model_path="data/kftt-data-1.0/run/model_step_15000.pt"

checkpoint = torch.load(model_path,map_location=lambda storage, loc: storage)

model_opt = ArgumentParser.ckpt_model_opts(checkpoint['opt'])
ArgumentParser.update_model_opts(model_opt)
ArgumentParser.validate_model_opts(model_opt)
fields = checkpoint['vocab']

# Avoid functionality on inference
model_opt.update_vocab = False

model = onmt.model_builder.build_base_model(model_opt, fields, use_gpu(opt), checkpoint,
                            opt.gpu)
if opt.fp32:
    model.float()
elif opt.int8:
    if opt.gpu >= 0:
        raise ValueError(
            "Dynamic 8-bit quantization is not supported on GPU")
    torch.quantization.quantize_dynamic(model, inplace=True)
model.eval()
model.generator.eval()
"""

Sequential(
  (0): Linear(in_features=512, out_features=12203, bias=True)
  (1): Cast()
  (2): LogSoftmax(dim=-1)
)

In [21]:
model

NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(13639, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_features=512, bias=True)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, o

In [31]:
from onmt.constants import DefaultTokens
from onmt.utils.misc import tile, set_random_seed, report_matrix

# アテンションデバッグをON
attn_debug=True
for batch in data_iter:
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=[src_vocab],
        attn_debug=attn_debug)
    translations = builder.from_batch(trans_batch)
    for trans in translations:
        print(trans.log(0))
        
        if attn_debug:
            preds = trans.pred_sents[0]
            preds.append(DefaultTokens.EOS)
            attns = trans.attns[0].tolist()
            if translator.data_type == "text":
                srcs = trans.src_raw
            else:
                srcs = [str(item) for item in range(len(attns[0]))]
            output = report_matrix(srcs, preds, attns)
            if translator.logger:
                translator.logger.info(output)
            else:
                #os.write(1, output.encode("utf-8"))
                print(output)
    break

NameError: ignored

In [34]:
builder = onmt.translate.TranslationBuilder(data=dataset, fields=vocab_fields)

In [37]:
from onmt.constants import DefaultTokens
from onmt.utils.misc import tile, set_random_seed, report_matrix

# アテンションデバッグをON
attn_debug=True
for batch in data_iter:
    batch_data = translator.translate_batch(
        batch, dataset.src_vocabs, attn_debug
    )
    translations = builder.from_batch(batch_data)

    for trans in translations:
        print(trans.log(0))
        
        if attn_debug:
            preds = trans.pred_sents[0]
            preds.append(DefaultTokens.EOS)
            attns = trans.attns[0].tolist()
            if translator.data_type == "text":
                srcs = trans.src_raw
            else:
                srcs = [str(item) for item in range(len(attns[0]))]
            output = report_matrix(srcs, preds, attns)
            if translator.logger:
                translator.logger.info(output)
            else:
                #os.write(1, output.encode("utf-8"))
                print(output)
    break

[2021-11-07 07:21:11,285 INFO]               Infobox    Buddhis 
        仏教 *0.5843355  0.4156645 
        用語 *0.9486390  0.0513610 
      </s> *0.9673884  0.0326116 

[2021-11-07 07:21:11,287 INFO]                 Dogen        was          a        Zen       monk         in        the      early    Kamakur     period          . 
        道元  0.0093218  0.0463321  0.0500570  0.1755526  0.0118073  0.0055185  0.0002921  0.0005321  0.0067596  0.0001872 *0.6936397 
         は  0.0019194  0.0288408  0.0649257  0.0809890  0.0503864  0.0177336  0.0022550  0.0003598  0.0263847  0.0007763 *0.7254291 
        鎌倉  0.0076908  0.1154914  0.1769026  0.1894506  0.0145433  0.0271018  0.0070224  0.0105953  0.0028024  0.0022899 *0.4461096 
        時代  0.0121528  0.0490180  0.0410148  0.1046753  0.0237850  0.0067809  0.0008257  0.0022857  0.0028120  0.0024362 *0.7542137 
        初期  0.0100437  0.1463979  0.1431923  0.1536881  0.0408283  0.0317716  0.0045548  0.0061332  0.0052312  0.0059326 *0.4522263 
   

# アテンションの利用

In [23]:
import onmt
import onmt.inputters
import onmt.translate
import onmt.model_builder
from collections import namedtuple

# Load the model.
Opt = namedtuple('Opt', ['model', 'data_type', 'reuse_copy_attn', "gpu"])

opt = Opt("/Users/esalesky/projects/models/lstm_clean_acc_51.81_ppl_15.58_e13.pt", "text",False, 0)
